In [1]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
from mrtool import MRData, LinearCovModel
from gkmodel import OverallModel, StudyModel, TwoStageModel, StagewiseModel, result_to_df

In [2]:
indicator = 'smoking'
df = pd.read_csv(f'../data/{indicator}.as.csv')

In [3]:
df.head(3)

,Unnamed: 0,location_id,year_id,sex_id,smoking,sdi,smoking_logit,ls_id,smoking_se,smoking_logit_se
0,1,10,1990,2,0.023706,0.264201,-3.718032,102,0.090516,1.269764
1,2,10,1990,1,0.283392,0.264201,-0.927695,101,0.090516,1.269764
2,3,10,1991,2,0.023371,0.269285,-3.732612,102,0.090516,1.269764


In [4]:
data_stage1 = MRData()
data_stage1.load_df(
    df,
    col_obs=f'{indicator}_logit',
    col_obs_se=f'{indicator}_logit_se',
    col_covs=['sdi', 'year_id'],
    col_study_id='ls_id'
)

In [5]:
cov_models1 = [
    LinearCovModel('intercept'),
    LinearCovModel('sdi',
                   use_spline=True,
                   spline_knots=np.linspace(0.0, 1.0, 5),
                   spline_l_linear=True,
                   spline_r_linear=True)
]
cov_models2 = [LinearCovModel('intercept'), LinearCovModel('year_id')]

### Using Overall Model and StudyModel separately

In [6]:
data_stage1

number of observations: 12240
number of covariates  : 3
number of studies     : 408

In [7]:
model_stage1 = OverallModel(data_stage1, cov_models1)

In [8]:
model_stage1.fit_model()

In [9]:
model_stage1.soln

array([-2.92576172,  0.15923559,  0.26159606,  0.74561337,  0.86764338])

In [10]:
model_stage1.data._sort_by_data_id()
df_stage1 = result_to_df(model_stage1, model_stage1.data, prediction='pred_stage1', residual='resi_stage1')
df_stage1 = pd.merge(df, df_stage1[['pred_stage1', 'resi_stage1']], left_index=True, right_index=True)

In [11]:
df_stage1.head()

,Unnamed: 0,location_id,year_id,sex_id,smoking,sdi,smoking_logit,ls_id,smoking_se,smoking_logit_se,pred_stage1,resi_stage1
0,1,10,1990,2,0.023706,0.264201,-3.718032,102,0.090516,1.269764,-3.379429,-0.338602
1,2,10,1990,1,0.283392,0.264201,-0.927695,101,0.090516,1.269764,-3.379429,2.451734
2,3,10,1991,2,0.023371,0.269285,-3.732612,102,0.090516,1.269764,-3.364603,-0.368009
3,4,10,1991,1,0.279966,0.269285,-0.944631,101,0.090516,1.269764,-3.364603,2.419972
4,5,10,1992,2,0.023096,0.274936,-3.744748,102,0.090516,1.269764,-3.348126,-0.396622


In [12]:
data_stage2 = MRData()
data_stage2.load_df(
    df_stage1,
    col_obs='resi_stage1',
    col_obs_se=f'{indicator}_logit_se',
    col_covs=['year_id'],
    col_study_id='ls_id'
)

In [13]:
model_stage2 = StudyModel(data_stage2, cov_models2)

In [14]:
model_stage2.fit_model()

### Using TwoStage Model

In [15]:
cov_models1 = [
    LinearCovModel('intercept'),
    LinearCovModel('sdi',
                   use_spline=True,
                   spline_knots=np.linspace(0.0, 1.0, 5),
                   spline_l_linear=True,
                   spline_r_linear=True)
]
cov_models2 = [LinearCovModel('intercept'), LinearCovModel('year_id')]

In [16]:
tsmodel = TwoStageModel(data_stage1, cov_models1, cov_models2)

In [17]:
tsmodel.fit_model()

In [18]:
result_to_df(tsmodel, tsmodel.data1)

,obs,obs_se,study_id,sdi,year_id,intercept,prediction,residual
0,-3.718032,1.269764,102,0.264201,1990,1.0,-3.690576,-0.027455
1,-0.927695,1.269764,101,0.264201,1990,1.0,-0.909230,-0.018465
2,-3.732612,1.269764,102,0.269285,1991,1.0,-3.699147,-0.033466
3,-0.944631,1.269764,101,0.269285,1991,1.0,-0.918248,-0.026383
4,-3.744748,1.269764,102,0.274936,1992,1.0,-3.706066,-0.038682
...,...,...,...,...,...,...,...,...
12235,-1.355529,1.269764,991,0.689284,2017,1.0,-1.306549,-0.048981
12236,-1.945212,1.269764,992,0.694041,2018,1.0,-1.860277,-0.084934
12237,-1.368356,1.269764,991,0.694041,2018,1.0,-1.309219,-0.059137
12238,-1.962946,1.269764,992,0.697889,2019,1.0,-1.859334,-0.103612


### Using StageWise

In [19]:
cov_models1 = [
    LinearCovModel('intercept'),
    LinearCovModel('sdi',
                   use_spline=True,
                   spline_knots=np.linspace(0.0, 1.0, 5),
                   spline_l_linear=True,
                   spline_r_linear=True)
]
cov_models2 = [LinearCovModel('intercept'), LinearCovModel('year_id')]

In [20]:
swmodel = StagewiseModel(data_stage1, [('overall', cov_models1), ('study', cov_models2)])

In [21]:
swmodel.fit_model()

In [22]:
swmodel.write_soln(0)

,name,value
0,intercept_0,-2.925762
1,sdi_0,0.159236
2,sdi_1,0.261596
3,sdi_2,0.745613
4,sdi_3,0.867643


In [23]:
swmodel.write_soln(1)

,study_id,intercept,year_id
0,61,64.634108,-0.031416
1,62,44.059996,-0.022703
2,71,15.818749,-0.007034
3,72,4.171925,-0.002693
4,81,73.292358,-0.036175
...,...,...,...
403,4222,44.931717,-0.023004
404,4351,39.034015,-0.019060
405,4352,33.734692,-0.017592
406,5221,59.673663,-0.029013


In [24]:
result_to_df(swmodel, swmodel.datas[0])

,obs,obs_se,study_id,sdi,year_id,intercept,prediction,residual
0,-3.718032,1.269764,102,0.264201,1990,1.0,-3.690576,-0.027455
1,-0.927695,1.269764,101,0.264201,1990,1.0,-0.909230,-0.018465
2,-3.732612,1.269764,102,0.269285,1991,1.0,-3.699147,-0.033466
3,-0.944631,1.269764,101,0.269285,1991,1.0,-0.918248,-0.026383
4,-3.744748,1.269764,102,0.274936,1992,1.0,-3.706066,-0.038682
...,...,...,...,...,...,...,...,...
12235,-1.355529,1.269764,991,0.689284,2017,1.0,-1.306549,-0.048981
12236,-1.945212,1.269764,992,0.694041,2018,1.0,-1.860277,-0.084934
12237,-1.368356,1.269764,991,0.694041,2018,1.0,-1.309219,-0.059137
12238,-1.962946,1.269764,992,0.697889,2019,1.0,-1.859334,-0.103612


### Compare Fit

In [25]:
print(model_stage1.soln - tsmodel.model1.soln)
print(model_stage1.soln - swmodel.fitted_models[0].soln)

[-1.77635684e-15 -7.77156117e-16  6.93889390e-15 -3.33066907e-16
  8.88178420e-16]
[-1.77635684e-15 -7.77156117e-16  6.93889390e-15 -3.33066907e-16
  8.88178420e-16]


In [26]:
for ls_id, soln in model_stage2.soln.items():
    assert np.linalg.norm(soln - tsmodel.model2.soln[ls_id]) / np.linalg.norm(soln) < 1e-5
    assert np.linalg.norm(soln - swmodel.fitted_models[1].soln[ls_id]) / np.linalg.norm(soln) < 1e-5

### Compare prediction

- predictions from OverallModel + StudyModel

In [27]:
model_stage2.data._sort_by_data_id()
df_stage2 = result_to_df(model_stage2, model_stage2.data, prediction='pred_stage2', residual='resi_stage2')
df_stage2 = pd.merge(df_stage1, df_stage2[['pred_stage2', 'resi_stage2']], left_index=True, right_index=True)

In [28]:
df_stage2.head()

,Unnamed: 0,location_id,year_id,sex_id,smoking,sdi,smoking_logit,ls_id,smoking_se,smoking_logit_se,pred_stage1,resi_stage1,pred_stage2,resi_stage2
0,1,10,1990,2,0.023706,0.264201,-3.718032,102,0.090516,1.269764,-3.379429,-0.338602,-0.311147,-0.027455
1,2,10,1990,1,0.283392,0.264201,-0.927695,101,0.090516,1.269764,-3.379429,2.451734,2.470199,-0.018465
2,3,10,1991,2,0.023371,0.269285,-3.732612,102,0.090516,1.269764,-3.364603,-0.368009,-0.334543,-0.033466
3,4,10,1991,1,0.279966,0.269285,-0.944631,101,0.090516,1.269764,-3.364603,2.419972,2.446355,-0.026383
4,5,10,1992,2,0.023096,0.274936,-3.744748,102,0.090516,1.269764,-3.348126,-0.396622,-0.357940,-0.038682


In [29]:
prediction = df_stage2['pred_stage1'].values + df_stage2['pred_stage2'].values

- prediction from TwoStageModel

In [30]:
prediction_ts = tsmodel.predict()

In [31]:
np.linalg.norm(prediction_ts - prediction)

3.3053424538595315e-10

- prediction from stagewise model

In [32]:
prediction_sw = swmodel.predict()

In [33]:
np.linalg.norm(prediction_sw - prediction)

8.186713392897345e-12

- predict with quantile

In [34]:
tsmodel.predict(slope_quantile=dict(year_id=0.15))

array([-24.95407048, -21.2811884 , -24.97332579, ..., -31.8902088 ,
       -46.75780772, -31.91150371])

In [35]:
swmodel.predict(slope_quantile=dict(year_id=0.15))

array([-24.95407048, -21.2811884 , -24.97332579, ..., -31.8902088 ,
       -46.75780772, -31.91150371])

### More plotting

In [ ]:
ls_id = 1021
index = df_stage2.ls_id == ls_id
obs = df_stage2[f'{indicator}_logit'].values[index]
year_id = df_stage2['year_id'].values[index]

prediction = df_stage2['pred_stage1'].values + df_stage2['pred_stage2'].values
print(prediction)
prediction = prediction[index]

plt.scatter(year_id, obs, label='observation')
plt.scatter(year_id, prediction, label='prediction')
plt.legend()
plt.title(f"loc_sex_id: {ls_id}")

In [ ]:
soln = np.vstack(list(model_stage2.soln.values()))

In [ ]:
plt.plot(np.sort(soln[:, 0]), np.linspace(0.0, 1.0, soln.shape[0]))
plt.title("random intercept cumulative density")

In [ ]:
plt.plot(np.sort(soln[:, 1]), np.linspace(0.0, 1.0, soln.shape[0]))
plt.title("random slope cumulative density")